# Malek et al. (2015) Model Code <a name="top" />

# Table of Contents
1. [Instructions](#instructions)
    1. [Parameter Optimization Against TSST Data Sets](#TSSTInstructions)
    2. [Parameter Optimization Against Basal Data Sets](#basalInstructions)
    3. [Running Without Parameter Optimization](#noOptInstructions)
2. [Imports](#imports)
2. [Parameters and Initial Conditions](#params)
3. [Put Raw Data into Arrays](#rawdata)
    1. [Plot Data Sets](#plotdata)
4. [Model Function--Includes ODE Solver](#modelfunction)
5. [Functions to Determine Delayed ACTH and Cortisol Values](#delays)
6. [Delay Functions for Larger Time Steps](#largerdelays)
7. [Cost Function Definition](#cost)
8. [Run the Optimization](#run)
9. [Save Output to File](#saveoutput)
10. [Compute Means and Std Devations of Parameters and Output as Table](#paramtable)
11. [Plots](#plots)
12. [No Optimization Run and Plot](#no-opt)
    1. [Plot the No-Optimization Simulation](#plotNoOpt)
13. [Dependencies](#dependencies)

## Instructions <a name="instructions"></a>

### Parameter Optimization Against TSST Data Sets <a name="TSSTInstructions" />

**Note:** To quickly run a cell (or a selection of cells), use the shortcut Shift+Enter (or you can also use the button labeled "Run" in the toolbar at the top).

To run simulations with parameter optimization against TSST data, there is no need to change any cells until the cost function cell. Simply run all cells up to the cell below the heading **Cost Function Definition**.

In order to run against a patient from the TSST data sets, simply change the second and fourth arguments to reflect the patient number and subject group. The subject groups are:

- nelson.melancholicACTH & nelson.melancholicCortisol (15 patients)
- nelson.atypicalACTH & nelson.atypicalCortisol (14 patients)
- nelson.neitherACTH & nelson.neitherCortisol (14 patients)
- nelson.healthyACTH & nelson.healthyCortisol (15 patients)

You could also run against the mean of all patients cortisol and ACTH concentration values by using `nelson.ACTH[:,1]` and `nelson.cortisol[:,1]`. Or you can run against the mean of any subgroup using `nelson.<subgroup name>Cortisol_mean[:,1]` and `nelson.<subgroup name>ACTH_mean[:,1]` (for instance `nelson.melancholicCortisol_mean[:,1]` & `nelson.melancholicACTH_mean[:,1]`). 

Note that the first column in each data set is the time steps, so indexing with `[:,0]` is referring to the time. These are the values we need to pass as the first (ACTH time steps) and third (cortisol time steps) arguments to the cost function.

The following are several examples of arguments you could pass the cost function with explanations:

- `nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,1], nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,1], simData`
    - The 1st patient in the Melancholic subgroup
- `nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,14], nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,14], simData`
    - The 14th patient in the Atypical subgroup
- `nelson.healthyACTH[:,0], nelson.healthyACTH[:,2], nelson.healthyCortisol[:,0], nelson.healthyCortisol[:,2], simData`
    - The 2nd patient in the Healthy Control group
- `nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1], simData`
    - The mean data set for all patients (depressed and control)
- `nelson.healthyACTH_mean[:,0], nelson.healthyACTH_mean[:,1], nelson.healthyCortisol_mean[:,0], nelson.healthyCortisol_mean[:,1], simData`
    - The mean of all control patients
    
In order to find out the actual Patient ID of the patient you are matching, try creating a new cell and entering the following command (using the subtype name and index number you used in the cost function arguments):

    print(melancholic_ids[1])
    
Next, you need to set the initial conditions for each ODE. In the cell directly below the heading **Run the Optimization**, you'll need to change the following line:

    y0 = [0, 0, 0, nelson.ACTH[0,1], nelson.cortisol[0,1]]
    
Depending on which data set you used, change `nelson.ACTH` and `nelson.cortisol` to match. Then, depending on which patient you used, change `[0,1]` in both places to `[0,#]` where # is replaced with the number from above. 

Alternatively, if you used the mean of all patients, you'd leave it unchanged.
    
And for each of the subgroup means, you'd use (or simply replace Healthy with whichever other subgroup you used):

    y0 = [0, 0, 0, nelson.healthyACTH_mean[0,1], nelson.healthyCortisol_mean[0,1]]
    
At this point, you are ready to run the optimization, so simply run the cells up to the heading **Save Output to File**. This may take some time, so while it is running you can move on to the next steps (if you run a cell while another is processing, it will add it to a queue).

The next few cells under the heading **Save Output to File** can be changed so that the filenames match the subgroup and patient you are running. For instance, the filename below is for the Melancholic subgroup patient with ID 3:

    df_t.to_excel(excel_writer = 'malekModel_output/malekModel-nelson-melancholic-patientID_3-timeSteps-5-
        iterations.xlsx')
    
The final step after saving the outputs is to plot the simulations against the real-world data. The cell under the heading **Plots** contains the code for this purpose. The lines of concern to ensure that you're showing the data set that you matched are (marked with comments in the actual code cell):
```
ax4.plot(nelson.ACTH[:,0], nelson.ACTH[:,1], label = "Nelson ACTH Data - Patient Mean", color = "orange")
ax5.plot(nelson.cortisol[:,0], nelson.cortisol[:,1], label = "Nelson Cortisol Data - Patient Mean", 
    color = "orange")
```
Here, change the first two arguments of each line to match exactly the arguments you used for the cost function. The labels can be changed to the patient subgroup and Patient ID you matched, also. And the filename for the figure can be changed in the last line of the cell:

    plt.savefig("malekModel_output/malekModel-nelson-melancholic-patientID_3-5-iterations-all-params-
        normalizedCost.png", dpi = 300)
    
### Parameter Optimization Against Basal Data Sets <a name="basalInstructions" />

Since these data sets have data points over a 24-hour period (1440 minutes), rather than 140.5 minutes, you will need to change the time interval over which the ODE solver integrates. To do this, go to the cell directly above the heading **Put Raw Data Into Arrays** and uncomment (delete the # at the start of the line) the lines:

    t_start = -0.5
    t_end = 1455.5
    t_step = 0.5

You'll need to comment out the other definitons for these variables (place a # at the start of the line).

The reason you add the extra 15 minutes is that you need to make sure that when you interpolate between your simulated data points the line covers every real-world data point so that you don't cause issues when computing the cost function (and the last data point for the Golier cortisol concentration data sets is at 1455 minutes).

After making this change, you need to again change the cost function arguments so that you are matching the basal data set in which you are interested. This time, however, you will also need to change the first and third arguments, because we need to tell the function the correct time steps for the data set.

First, choose which data set you wish to match. Here are the options:

- yehuda.controlCortisol
- yehuda.PTSDCortisol
- yehuda.depressedCortisol
- carroll.controlCortisol & carroll.controlACTH
- carroll.LCDepressedCortisol & carroll.LCDepressedACTH (LC = Low Cortisol)
- carroll.HCDepressedCortisol & carroll.HCDepressedACTH (HC = High Cortisol)
- golier.PTSDCortisol & golierPTSDACTH
- golier.nonPTSDTraumaExposedCortisol & golier.nonPTSDTraumaExposedACTH
- golier.nonPTSDNonExposedCortisol & golier.nonPTSDNonExposedACTH
- bremner.abusedPTSDCortisol
- bremner.nonAbusedPTSDCortisol
- bremner.nonAbusedNonPTSDCortisol

**Note:** To see what any of these data sets looks like, click on the **Plot Basal Data Sets** heading in the Table of Contents.

**Note Also:** These data sets all come in smoothed versions (each data point is set to the average of the nearest 5 points of the unsmoothed data). Also, the data sets by Carroll, Golier and Bremner also come in rearranged (or smoothed & rearranged) versions to match the starting time of the Yehuda data (10AM). To use any of these versions, simply append one of the following tags to the end of the data set name (before the indices): `_smooth`, `_rearr`, or `_rearr_smooth`.

First, I will cover what to do with data sets that contain both ACTH and cortisol values, and then afterwards I will cover using the Yehuda and Bremner data sets (which have only cortisol concentration data). For all of these data sets, the first column is the time step values. This means that if you take any of these arrays and index it with `[:,0]`, you are referring to the time steps. These are the values we need to pass as the first (ACTH time steps) and third (cortisol time steps) arguments to the cost function.

Then for the second and fourth arguments, you index the same data sets with `[:,1]` to mean the second column (which contains the mean concentration values for each patient group).

Here are a couple of examples showing arguments you can pass to the cost function:

- `carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1], simData`
    - The smoothed Control group mean for the Carroll data set
- `golier.nonPTSDTraumaExposedACTH[:,0], golier.nonPTSDTraumaExposedACTH[:,1], golier.nonPTSDTraumaExposedCortisol[:,0], golier.nonPTSDTraumaExposedCortisol[:,1], simData`
    - The Trauma-Exposed Control group mean for the Golier data set
    
In order to run simulations against data sets that do not include ACTH concentration data, you will need to change the name of the function to `costFun.cost_noACTH` and then simply do not include the two arguments for ACTH data. To use the Yehuda Control group data, this would look like:

    return costFun.SSE_cost_noACTH(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], simData)

You'll also need to change the initial conditions in the cell below the heading **Run the Optimization**, again. Use the name of the data set you used for the cost function, and change the indices to `[0,1]` for both. Here's an example (using the smoothed Carroll Control group data set):

    y0 = [0, 0, 0, carroll.controlACTH_smooth[0,1], carroll.controlCortisol_smooth[0,1]]
    
Next, because you changed the length of the interval over which the ODE solver integrates, you'll need to change the arrays initialized in the next cell to be longer. For each of the last 4 lines in the cell, you'll need to replace the number 282 with 2882 (since we start at -0.5 and go until 1440.5 with a step size of 0.5, that's 2882 steps total).

At this point, you're ready to run the parameter optimization, so run the next cell. Again, it may take a while, so you can start editing the remaining cells while you wait.

The cells under the heading **Save Output to File** should each have the filename changed to something that reflects the data set you're matching now. For instance, the Excel filename for sims_acth when matching the smoothed Carroll Control group would become:

    df_crh.to_excel(excel_writer = 'sriramModel_output/sriramModel-carroll-control-smooth-sims-crh-5-
        iterations.xlsx')
            
Finally, the cell under the heading **Plots** needs to have the same lines changed as described above. For instance, when matching the smoothed Carroll Control group, they would become:
```
ax2.plot(carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], label = "Carroll Control ACTH Data", 
    color = "orange")
ax3.plot(carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1], label = "Carroll Control 
    Cortisol Data", color = "orange")
```
The data set names and indices here should match exactly the names and indices used as arguments to the cost function. Also, the last line of the cell can have the filename changed as described above to reflect which data set you are plotting.

### Running Without Parameter Optimization <a name="noOptInstructions" />

To run the model with any set of paramaters you desire, without optimization, you can use the cells under the heading **No Optimization Run**. Set the parameters, initial conditions and time interval you want to use by changing the values defined in the section **Parameters and Initial Conditions**. Then you can either run the cell immediately after the heading **Plot the No-Optimization Simulation** to plot all 5 variables, including the inflammatory cytokines, or you can run the second cell to plot only the HPA axis variables (ACTH and cortisol).

[Back to Top](#top)

## Imports <a name="imports"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sco
from scipy import optimize
from scipy.interpolate import interp1d
import mpld3
from tabulate import tabulate
import pandas as pd
from HPAmodeling import DEsolver, costFun
from HPAmodeling.dataImport import data

[Back to Top](#top)

## Parameters and Initial Conditions <a name="params"></a>

In [ ]:
# initial conditions
# order is: LPS endotoxin, TNF-alpha, IL-6, ACTH, Cortisol

y0 = [0, 0, 0,14.47489,1.903323]

In [ ]:
# authors' published parameters

e_P = 0.05
e_T = 0.038
e_S = 0.02
e_A = 0.04
e_C = 0.01
d_1 = 0.026
d_2 = 0.068
d_3 = 0.063
d_4 = 2.37
d_5 = 9.39
d_6 = 0.35
k = 0.0504
tau_1 = 10
tau_2 = 10
m_1 = 4
m_2 = 4
c = 6.11
a = 21
h = 7.66
alpha = 0.28
x_1 = 3.25
x_2 = 0.86
x_3 = 0.016
x_4 = 6.11
x_5 = 1.39
x_6 = 1.57
x_7 = 6.11
x_8 = 1.72
x_9 = 0.87
x_10 = 0.94
x_11 = 1.87
x_12 = 1.97

authors_params = [e_P, e_T, e_S, e_A, e_C, d_1, d_2, d_3, d_4, d_5, d_6, k, tau_1, tau_2, m_1, m_2, c, a, h, alpha, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12]

In [ ]:
# compute bounds based on +- 10%
bound = x_7
print(bound - bound*.1)
print(bound + bound*.1)

In [ ]:
# bounds for parameter optimization
# starting with +- 10% since we do not have published ranges in the paper
# order is: e_P, e_T, e_S, e_A, e_C, d_1, d_2, d_3, d_4, d_5, d_6, k, tau_1, tau_2, m_1, m_2, c, a, h, alpha, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12
# the authors included 95% CI for several of the parameters, so we have used those intervals as the bounds for
#     the parametrs they included
# the ones included are: x_1, x_2, x_3, x_5, x_6, x_8, x_9, x_10, x_11, x_12, k, d_1, d_2, d_3, d_4, d_5, d_6
bounds = [(0.045, 0.055), (0.0342, 0.0418), (0.018, 0.022), (0.036, 0.044), (0.009, 0.011), (0.021, 2.21), (0.052, 1.39), (0.0074, 0.3), (0.71, 2.63), (8.59, 13.55), (0.27, 2.46), (0.03, 0.54), (9., 11.), (9., 11.), (3.6, 4.4), (3.6, 4.4), (5.499, 6.721), (18.9, 23.1), (6.894, 8.426), (0.252, 0.308), (2.62, 6.13), (0.09, 2.02), (0.013, 3.4), (5.499, 6.721), (0.81, 3.79), (2.81, 5.68), (4.499, 6.721), (0.99, 5.16), (0.27, 4.07), (0.05, 2.6), (0.2, 5.43), (1.47, 6.45)]


In [ ]:
# define time interval for integration

# time interval and step definition
# all data sets end on 1440.0 or earlier except the Golier cortisol sets,
# they end on 1455.0, so I should set t_end = 1455.01 when matching them
#t_start = -0.01
#t_end = 1455.01
#t_step = 0.01

# for matching Nelson data, use these values of t_start, t_end and t_step
t_start = -0.5
t_end = 140.5
t_step = 0.5

[Back to Top](#top)

## Put Raw Data into Arrays <a name="rawdata"></a>

In [ ]:
# Create an instance of the data class for each data set contained in the HPAmodeling library, and set the time
# scale to hours.
yehuda = data("yehuda", "minutes")
carroll = data("carroll", "minutes")
golier = data("golier", "minutes")
bremner = data("bremner", "minutes")
nelson = data("nelson", "minutes")

### Plot Data Sets <a name="plotdata"></a>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], label = "Control Group Cortisol")
ax1.plot(yehuda.controlCortisol_smooth[:,0], yehuda.controlCortisol_smooth[:,1], label = "Control Group Cortisol - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(yehuda.PTSDCortisol[:,0], yehuda.PTSDCortisol[:,1], label = "PTSD Group Cortisol")
ax2.plot(yehuda.PTSDCortisol_smooth[:,0], yehuda.PTSDCortisol_smooth[:,1], label = "PTSD Group Cortisol - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(yehuda.depressedCortisol[:,0], yehuda.depressedCortisol[:,1], label = "Depression Group Cortisol")
ax3.plot(yehuda.depressedCortisol_smooth[:,0], yehuda.depressedCortisol_smooth[:,1], label = "Depression Group Cortisol - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
#mpld3.enable_notebook()
%matplotlib inline

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4, figsize = (15,15))

ax1.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax1.plot(carroll.HCDepressedCortisol_rearr[:,0], carroll.HCDepressedCortisol_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax1.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax1.plot(carroll.HCDepressedCortisol_rearr_smooth[:,0], carroll.HCDepressedCortisol_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="upper right", shadow = True, fancybox = True)

ax2.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax2.plot(carroll.LCDepressedCortisol_rearr[:,0], carroll.LCDepressedCortisol_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax2.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax2.plot(carroll.LCDepressedCortisol_rearr_smooth[:,0], carroll.LCDepressedCortisol_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="upper right", shadow = True, fancybox = True)

ax3.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax3.plot(carroll.HCDepressedACTH_rearr[:,0], carroll.HCDepressedACTH_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax3.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax3.plot(carroll.HCDepressedACTH_rearr_smooth[:,0], carroll.HCDepressedACTH_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax3.legend(loc="upper right", shadow = True, fancybox = True)

ax4.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax4.plot(carroll.LCDepressedACTH_rearr[:,0], carroll.LCDepressedACTH_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax4.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax4.plot(carroll.LCDepressedACTH_rearr_smooth[:,0], carroll.LCDepressedACTH_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="upper right", shadow = True, fancybox = True)

In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(nrows = 6, figsize = (15,20))

ax1.plot(golier.PTSDCortisol_rearr_smooth[:,0], golier.PTSDCortisol_rearr_smooth[:,1], label = "Trauma Exposed PTSD Cortisol - Smoothed")
ax1.plot(golier.PTSDCortisol_rearr[:,0], golier.PTSDCortisol_rearr[:,1], label = "Trauma Exposed PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,0], golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD Cortisol - Smoothed")
ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr[:,0], golier.nonPTSDTraumaExposedCortisol_rearr[:,1], label = "Trauma Exposed Non-PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(golier.nonPTSDNonExposedCortisol_rearr_smooth[:,0], golier.nonPTSDNonExposedCortisol_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD Cortisol - Smoothed")
ax3.plot(golier.nonPTSDNonExposedCortisol_rearr[:,0], golier.nonPTSDNonExposedCortisol_rearr[:,1], label = "Non-Exposed Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)

ax4.plot(golier.PTSDACTH_rearr_smooth[:,0], golier.PTSDACTH_rearr_smooth[:,1], label = "Trauma Exposed PTSD ACTH - Smoothed")
ax4.plot(golier.PTSDACTH_rearr[:,0], golier.PTSDACTH_rearr[:,1], label = "Trauma Exposed PTSD ACTH")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="lower right", shadow = True, fancybox = True)

ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,0], golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD ACTH - Smoothed")
ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr[:,0], golier.nonPTSDTraumaExposedACTH_rearr[:,1], label = "Trauma Exposed Non-PTSD ACTH")
ax5.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax5.legend(loc="lower right", shadow = True, fancybox = True)

ax6.plot(golier.nonPTSDNonExposedACTH_rearr_smooth[:,0], golier.nonPTSDNonExposedACTH_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD ACTH - Smoothed")
ax6.plot(golier.nonPTSDNonExposedACTH_rearr[:,0], golier.nonPTSDNonExposedACTH_rearr[:,1], label = "Non-Exposed Non-PTSD ACTH")
ax6.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax6.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(bremner.abusedPTSDCortisol_rearr_smooth[:,0], bremner.abusedPTSDCortisol_rearr_smooth[:,1], label = "Abused PTSD Cortisol - Smoothed")
ax1.plot(bremner.abusedPTSDCortisol_rearr[:,0], bremner.abusedPTSDCortisol_rearr[:,1], label = "Abused PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(bremner.nonAbusedPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused PTSD Cortisol - Smoothed")
ax2.plot(bremner.nonAbusedPTSDCortisol_rearr[:,0], bremner.nonAbusedPTSDCortisol_rearr[:,1], label = "Non-Abused PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused Non-PTSD Cortisol - Smoothed")
ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr[:,0], bremner.nonAbusedNonPTSDCortisol_rearr[:,1], label = "Non-Abused Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax3.legend(loc="lower left", shadow = True, fancybox = True)


In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15, 15))

ax1.plot(nelson.ACTH[:,0], nelson.ACTH[:,1])
ax2.plot(nelson.cortisol[:,0], nelson.cortisol[:,1])

[Back to Top](#top)

## Model Function -- Includes ODE Solver <a name="modelfunction"></a>

In [ ]:
def model(params, ics):
    # function containing the system, to be called by solver
    def ode_system(t, y):
        # initialize array to hold ODE function values
        dy = np.zeros(5)
        #dy = np.zeros(2)
        
        # define parameter values
        [e_P, e_T, e_S, e_A, e_C, d_1, d_2, d_3, d_4, d_5, d_6, k, tau_1, tau_2, m_1, m_2, c, a, h, alpha, x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8, x_9, x_10, x_11, x_12] = params

        # the ODE system itself
        dy[0] = -e_P*y[0] - d_1*(y[1]/(x_1 + y[1]))*(y[2]/(x_2 + y[2]))*y[0]
        dy[1] = -e_T*y[1] + k*(y[0]/(x_3 + y[0])) + ((d_2/(x_4 + y[4])) + (d_3/(x_5 + y[2])))*(y[1]/(x_6 + y[1]))
        dy[2] = -e_S*y[2] + d_4*(1/(x_7 + y[4]))*(y[1]/(x_8 + y[1]))
        # to include circadian input function, uncomment line with E(t) included in ACTH
        #dy[3] = -e_A*y[3] + E(t)*h*((c**m_1)/(c**m_1 + DEsolver.delayedCORT**m_1)) + d_5*(y[2]/(x_9 + y[2]))*(y[1]/(x_10 + y[1]))
        dy[3] = -e_A*y[3] + h*((c**m_1)/(c**m_1 + DEsolver.delayedCORT**m_1)) + d_5*(y[2]/(x_9 + y[2]))*(y[1]/(x_10 + y[1]))
        dy[4] = -e_C*y[4] + alpha*((DEsolver.delayedACTH**m_2)/(DEsolver.delayedACTH**m_2 + a**m_2)) + d_6*(y[2]/(x_11 + y[2]))*(y[1]/(x_12 + y[1]))
        
        return dy
    
    def E(t):
        return (0.6 + 0.171*np.sin(((2*np.pi)/(24*60))*t) - 0.4698*np.cos(((2*np.pi)/(24*60))*t))
    
    # Call the solve() function from my DEsolver module, and pass all of the information it needs.
    # Arguments are as follows: ODE function to solve, array of initial conditions, start time, step size, end time, 
    # The last five arguments are optional (leave blank for ODE systems) for delay differential equation systems,
    #  tau1 is the delay in ACTH, tau2 is the delay in CORT, and delay is an array of booleans to set whether 
    #  we use delays in [CRH, ACTH, CORT]
    timeSeries = DEsolver.solve(ode_system, ics, t_start, t_step, t_end, tau1 = 10, tau2 = 10, delay = [False, True, True], delay_rough = True)
    return timeSeries

[Back to Top](#top)

## Cost Function Definition <a name="cost"></a>

In [ ]:
def cost_fun(params):
    simData = model(params, y0)
    
    return costFun.SSE_cost(nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1], simData)

[Back to Top](#top)

## Run the Optimization <a name="run"></a>

In [ ]:
# initial conditions
# order: LPS endotoxin, TNF-alpha, IL-6, ACTH, CORT

# based on: golierPTSD
y0 = [0, 0, 0, nelson.ACTH[0,1], nelson.cortisol[0,1]]


In [ ]:
# number of times to run the optimization
n = 5

# define an array to hold the population of parameter vectors
opt_pars = np.zeros((n, len(bounds)+1))

# initialize arrays to save simulation cortisol and ACTH data from each optimization
sims_cort = np.zeros((282, n))
sims_acth = np.zeros((282, n))
sims_il6 = np.zeros((282, n))
sims_tnf = np.zeros((282, n))
sims_lps = np.zeros((282, n))

In [ ]:
%%time

# loop n times, running the optimization each time
for i in range(0,n):
    
    print(f"Optimization Run #{i+1}")
    
    # call the differential evolution optimization function on the cost function
    res = sco.differential_evolution(cost_fun, bounds, maxiter = 250, disp = True, popsize = 1)
    
    # alternatively, we can run the SHGO algorithm with the sampling_method = "sobol" flag to do global
    #     optimization with reporting all local minima, as well
    #res = sco.shgo(cost_fun, bounds, callback=callback_fun(*shgo_iter_steps), options = {"f_min": 0.1, "maxiter": None, "minimize_every_iter": True, "local_iter": False, "disp": True}, iters = 3)
    #res = sco.basinhopping(cost_fun, x0, niter = 1000)
    #res = sco.dual_annealing(cost_fun, bounds)
    
    # plug the optimized parameters into the solver
    optimizedSimData = model(res.x, y0)
    # save CRH, cortisol and ACTH data into sims arrays
    sims_cort[:,i] = optimizedSimData[:,5]
    sims_acth[:,i] = optimizedSimData[:,4]
    sims_il6[:,i] = optimizedSimData[:,3]
    sims_tnf[:,i] = optimizedSimData[:,2]
    sims_lps[:,i] = optimizedSimData[:,1]
    
    # save the cost function values and optimized parameters for each iteration into the array opt_pars
    opt_pars[i,0] = res.fun
    opt_pars[i,1:] = res.x

[Back to Top](#top)

## Save Output to File <a name="saveoutput"></a>

In [ ]:
df_t = pd.DataFrame(optimizedSimData[:,0])
df_cort = pd.DataFrame(sims_cort)
df_acth = pd.DataFrame(sims_acth)
df_il6 = pd.DataFrame(sims_il6)
df_tnf = pd.DataFrame(sims_tnf)
df_lps = pd.DataFrame(sims_lps)

df_t.to_excel(excel_writer = 'malekModel_output/malekModel-nelson-patient20-timeSteps-5-iterations-normalizedCost.xlsx')
df_cort.to_excel(excel_writer = 'malekModel_output/malekModel-nelson-patient20-sims-cort-5-iterations-normalizedCost.xlsx')
df_acth.to_excel(excel_writer = 'malekModel_output/malekModel-nelson-patient20-sims-acth-5-iterations-normalizedCost.xlsx')
df_il6.to_excel(excel_writer = 'malekModel_output/malekModel-nelson-patient20-sims-il6-5-iterations-normalizedCost.xlsx')
df_tnf.to_excel(excel_writer = 'malekModel_output/malekModel-nelson-patient20-sims-tnf-5-iterations-normalizedCost.xlsx')
df_lps.to_excel(excel_writer = 'malekModel_output/malekModel-nelson-patient20-sims-lps-5-iterations-normalizedCost.xlsx')


In [ ]:
np.savetxt('malekModel_output/malekModel-nelson-patient50-opt-pars-5-iterations-normalizedCost.txt', opt_pars)


In [ ]:
np.savetxt('malekModel_output/malekModel-nelson-patient50-sims-lps-5-iterations-normalizedCost.txt', sims_lps)
np.savetxt('malekModel_output/malekModel-nelson-patient50-sims-tnf-5-iterations-normalizedCost.txt', sims_tnf)
np.savetxt('malekModel_output/malekModel-nelson-patient50-sims-il6-5-iterations-normalizedCost.txt', sims_il6)
np.savetxt('malekModel_output/malekModel-nelson-patient50-sims-acth-5-iterations-normalizedCost.txt', sims_acth)
np.savetxt('malekModel_output/malekModel-nelson-patient50-sims-cort-5-iterations-normalizedCost.txt', sims_cort)

In [ ]:
np.savetxt('malekModel_output/malekModel-nelson-patient50-initial-conditions-5-iterations-normalizedCost.txt', y0)
np.savetxt('malekModel_output/malekModel-nelson-patient50-bounds-5-iterations-normalizedCost.txt', bounds)

## Compute Means and Std Devations of Parameters and Output as Table <a name="paramtable"></a>

In [ ]:
# compute parameter means and standard deviations
e_P_mean = np.mean(opt_pars[:,1])
e_P_std = np.std(opt_pars[:,1])
e_T_mean = np.mean(opt_pars[:,2])
e_T_std = np.std(opt_pars[:,2])
e_S_mean = np.mean(opt_pars[:,3])
e_S_std = np.std(opt_pars[:,3])
e_A_mean = np.mean(opt_pars[:,4])
e_A_std = np.std(opt_pars[:,4])
e_C_mean = np.mean(opt_pars[:,5])
e_C_std = np.std(opt_pars[:,5])
d_1_mean = np.mean(opt_pars[:,6])
d_1_std = np.std(opt_pars[:,6])
d_2_mean = np.mean(opt_pars[:,7])
d_2_std = np.std(opt_pars[:,7])
d_3_mean = np.mean(opt_pars[:,8])
d_3_std = np.std(opt_pars[:,8])
d_4_mean = np.mean(opt_pars[:,9])
d_4_std = np.std(opt_pars[:,9])
d_5_mean = np.mean(opt_pars[:,10])
d_5_std = np.std(opt_pars[:,10])
d_6_mean = np.mean(opt_pars[:,11])
d_6_std = np.std(opt_pars[:,11])
k_mean = np.mean(opt_pars[:,12])
k_std = np.std(opt_pars[:,12])
tau_1_mean = np.mean(opt_pars[:,13])
tau_1_std = np.std(opt_pars[:,13])
tau_2_mean = np.mean(opt_pars[:,14])
tau_2_std = np.std(opt_pars[:,14])
m_1_mean = np.mean(opt_pars[:,15])
m_1_std = np.std(opt_pars[:,15])
m_2_mean = np.mean(opt_pars[:,16])
m_2_std = np.std(opt_pars[:,16])
c_mean = np.mean(opt_pars[:,17])
c_std = np.std(opt_pars[:,17])
a_mean = np.mean(opt_pars[:,18])
a_std = np.std(opt_pars[:,18])
h_mean = np.mean(opt_pars[:,19])
h_std = np.std(opt_pars[:,19])
alpha_mean = np.mean(opt_pars[:,20])
alpha_std = np.std(opt_pars[:,20])
x_1_mean = np.mean(opt_pars[:,21])
x_1_std = np.std(opt_pars[:,21])
x_2_mean = np.mean(opt_pars[:,22])
x_2_std = np.std(opt_pars[:,22])
x_3_mean = np.mean(opt_pars[:,23])
x_3_std = np.std(opt_pars[:,23])
x_4_mean = np.mean(opt_pars[:,24])
x_4_std = np.std(opt_pars[:,24])
x_5_mean = np.mean(opt_pars[:,25])
x_5_std = np.std(opt_pars[:,25])
x_6_mean = np.mean(opt_pars[:,26])
x_6_std = np.std(opt_pars[:,26])
x_7_mean = np.mean(opt_pars[:,27])
x_7_std = np.std(opt_pars[:,27])
x_8_mean = np.mean(opt_pars[:,28])
x_8_std = np.std(opt_pars[:,28])
x_9_mean = np.mean(opt_pars[:,29])
x_9_std = np.std(opt_pars[:,29])
x_10_mean = np.mean(opt_pars[:,30])
x_10_std = np.std(opt_pars[:,30])
x_11_mean = np.mean(opt_pars[:,31])
x_11_std = np.std(opt_pars[:,31])
x_12_mean = np.mean(opt_pars[:,32])
x_12_std = np.std(opt_pars[:,32])

In [ ]:
# print a table of parameter means and standard deviations
print(tabulate([["e_P", "%f +- %f" % (e_P_mean, e_P_std)], ["e_T", "%f +- %f" % (e_T_mean, e_T_std)], ["e_S", "%f +- %f" % (e_S_mean, e_S_std)], ["e_A", "%f +- %f" % (e_A_mean, e_A_std)], ["e_C", "%f +- %f" % (e_C_mean, e_C_std)], ["d_1", "%f +- %f" % (d_1_mean, d_1_std)], ["d_2", "%f +- %f" % (d_2_mean, d_2_std)], ["d_3", "%f +- %f" % (d_3_mean, d_3_std)], ["d_4", "%f +- %f" % (d_4_mean, d_4_std)], ["d_5", "%f +- %f" % (d_5_mean, d_5_std)], ["d_6", "%f +- %f" % (d_6_mean, d_6_std)], ["k", "%f +- %f" % (k_mean, k_std)], ["tau_1", "%f +- %f" % (tau_1_mean, tau_1_std)], ["tau_2", "%f +- %f" % (tau_2_mean, tau_2_std)], ["m_1", "%f +- %f" % (m_1_mean, m_1_std)], ["m_2", "%f +- %f" % (m_2_mean, m_2_std)], ["c", "%f +- %f" % (c_mean, c_std)], ["a", "%f +- %f" % (a_mean, a_std)], ["h", "%f +- %f" % (h_mean, h_std)], ["alpha", "%f +- %f" % (alpha_mean, alpha_std)], ["x_1", "%f +- %f" % (x_1_mean, x_1_std)], ["x_2", "%f +- %f" % (x_2_mean, x_2_std)], ["x_3", "%f +- %f" % (x_3_mean, x_3_std)], ["x_4", "%f +- %f" % (x_4_mean, x_4_std)], ["x_5", "%f +- %f" % (x_5_mean, x_5_std)], ["x_6", "%f +- %f" % (x_6_mean, x_6_std)], ["x_7", "%f +- %f" % (x_7_mean, x_7_std)], ["x_8", "%f +- %f" % (x_8_mean, x_8_std)], ["x_9", "%f +- %f" % (x_9_mean, x_9_std)], ["x_10", "%f +- %f" % (x_10_mean, x_10_std)], ["x_11", "%f +- %f" % (x_11_mean, x_11_std)], ["x_12", "%f +- %f" % (x_12_mean, x_12_std)]], headers = ["Parameter", "Mean +- Standard Deviation"]))

In [ ]:
# save means and std devs to a file
np.savetxt('malekModel_output/malekModel-nelson-patient50-param-means-stds-5-iterations-normalizedCost.txt', [e_P_mean, e_P_std, e_T_mean, e_T_std, e_S_mean, e_S_std, e_A_mean, e_A_std, e_C_mean, e_C_std, d_1_mean, d_1_std, d_2_mean, d_2_std, d_3_mean, d_3_std, d_4_mean, d_4_std, d_5_mean, d_5_std, d_6_mean, d_6_std, k_mean, k_std, tau_1_mean, tau_1_std, tau_2_mean, tau_2_std, m_1_mean, m_1_std, m_2_mean, m_2_std, c_mean, c_std, a_mean, a_std, h_mean, h_std, alpha_mean, alpha_std, x_1_mean, x_1_std, x_2_mean, x_2_std, x_3_mean, x_3_std, x_4_mean, x_4_std, x_5_mean, x_5_std, x_6_mean, x_6_std, x_7_mean, x_7_std, x_8_mean, x_8_std, x_9_mean, x_9_std, x_10_mean, x_10_std, x_11_mean, x_11_std, x_12_mean, x_12_std])

[Back to Top](#top)

## Plots <a name="plots"></a>

In [ ]:
%matplotlib inline

# plot with 3 subfigures--one for CRH, one for ACTH and one for cortisol
#fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows = 5, figsize = (15, 25))
fig, (ax4, ax5) = plt.subplots(nrows = 2, figsize = (15, 15))

# if running more than one iteration of optimization, uncomment plot command with np.mean and fill_between
# and comment plot command without np.mean
#
# if only one run is stored in sims variables, comment out plot commands with np.mean and fill_between
#ax1.plot(optimizedSimData[:,0], sims_lps, label = "Simulated LPS Endotoxin Data", color = "blue")
#ax1.plot(optimizedSimData[:,0], np.mean(sims_lps, axis = 1), label = "Mean Simulated LPS Endotoxin", color = "blue")
#ax1.fill_between(optimizedSimData[:,0], np.mean(sims_lps, axis = 1) - np.std(sims_lps, axis = 1), np.mean(sims_lps, axis = 1) + np.std(sims_lps, axis = 1), alpha = 0.3, label = "Simulated LPS Endotoxin Standard Deviation")
#ax1.legend(loc = "upper left", shadow = True, fancybox = True)
#ax1.set(xlabel = "Time (min)", ylabel = "LPS Endotoxin (pg/mL)", title = "LPS Endotoxin Concentration")

#ax2.plot(optimizedSimData[:,0], sims_tnf, label = "Simulated TNF-alpha Data", color = "blue")
#ax2.plot(optimizedSimData[:,0], np.mean(sims_tnf, axis = 1), label = "Mean Simulated TNF-alpha", color = "blue")
#ax2.fill_between(optimizedSimData[:,0], np.mean(sims_tnf, axis = 1) - np.std(sims_tnf, axis = 1), np.mean(sims_tnf, axis = 1) + np.std(sims_tnf, axis = 1), alpha = 0.3, label = "Simulated TNF-alpha Standard Deviation")
#ax2.legend(loc = "upper left", shadow = True, fancybox = True)
#ax2.set(xlabel = "Time (min)", ylabel = "TNF-alpha (pg/mL)", title = "TNF-alpha Concentration")

#ax3.plot(optimizedSimData[:,0], sims_il6, label = "Simulated IL-6 Data", color = "blue")
#ax3.plot(optimizedSimData[:,0], np.mean(sims_il6, axis = 1), label = "Mean Simulated IL-6n", color = "blue")
#ax3.fill_between(optimizedSimData[:,0], np.mean(sims_il6, axis = 1) - np.std(sims_il6, axis = 1), np.mean(sims_il6, axis = 1) + np.std(sims_il6, axis = 1), alpha = 0.3, label = "Simulated IL-6 Standard Deviation")
#ax3.legend(loc = "upper left", shadow = True, fancybox = True)
#ax3.set(xlabel = "Time (min)", ylabel = "IL-6 (pg/mL)", title = "IL-6 Concentration")

# change this line when you change which data set you are matching
ax4.plot(nelson.ACTH[:,0], nelson.ACTH[:,1], label = "Nelson ACTH Data - Patient Mean", color = "orange")

# for only 1 iteration, uncomment below and comment the following line
#ax4.plot(optimizedSimData[:,0], sims_acth, label = "Simulated ACTH Data", color = "blue")
ax4.plot(optimizedSimData[:,0], np.mean(sims_acth, axis = 1), label = "Mean Simulated ACTH", color = "blue")
ax4.fill_between(optimizedSimData[:,0], np.mean(sims_acth, axis = 1) - np.std(sims_acth, axis = 1), np.mean(sims_acth, axis = 1) + np.std(sims_acth, axis = 1), alpha = 0.3, label = "Simulated ACTH Standard Deviation")
ax4.legend(loc = "upper left", shadow = True, fancybox = True)
ax4.set(xlabel = "Time (min)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

# change this line when you change which data set you are matching
ax5.plot(nelson.cortisol[:,0], nelson.cortisol[:,1], label = "Nelson Cortisol Data - Patient Mean", color = "orange")

# for only 1 iteration, uncomment below and comment the following line
#ax5.plot(optimizedSimData[:,0], sims_cort, label = "Simulated Cortisol Data", color = "blue")
ax5.plot(optimizedSimData[:,0], np.mean(sims_cort, axis = 1), label = "Mean Simulated Cortisol", color = "blue")
ax5.fill_between(optimizedSimData[:,0], np.mean(sims_cort, axis = 1) - np.std(sims_cort, axis = 1), np.mean(sims_cort, axis = 1) + np.std(sims_cort, axis = 1), alpha = 0.3, label = "Simulated Cortisol Standard Deviation")
ax5.legend(loc = "upper left", shadow = True, fancybox = True)
ax5.set(xlabel = "Time (min)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

# include the value of the cost function for this set of graphs at the bottom of the figure
txt = "cost = [2362.53, 2395.96]"
plt.figtext(0.1, 0.09, txt, wrap = True, horizontalalignment = 'center', fontsize = 12)
plt.savefig("malekModel_output/malekModel-nelson-patientMean-5-iterations-normalizedCost.png", dpi = 300)


[Back to Top](#top)

## No Optimization Run and Plot <a name="no-opt"></a>

In [ ]:
# shorten the time of iteration to see how this delay works
t_start = -0.5
t_end = 2880.5
t_step = 0.5

# set the initial conditions
y0 = [0, 0, 0, 24, 8]
#y0 = [24, 8]

In [ ]:
%%time

# run the solver with authors' published parameters
optimizedSimData = model(authors_params, y0)
    
# save CRH, cortisol and ACTH data into sims arrays
#sims_cort = optimizedSimData[:,2]
#sims_acth = optimizedSimData[:,1]

sims_cort = optimizedSimData[:,5]
sims_acth = optimizedSimData[:,4]
sims_il6 = optimizedSimData[:,3]
sims_tnf = optimizedSimData[:,2]
sims_lps = optimizedSimData[:,1]

[Back to Top](#top)

## Plot the No-Optimization Simulation <a name="plotNoOpt" />

In [ ]:
# plot all 5 variables, including the inflammatory cytokine concentrations

%matplotlib inline
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows = 5, figsize = (15,30))

ax1.plot(optimizedSimData[:,0], sims_lps, label = "Simulated LPS Endotoxin Data", color = "blue")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (min)", ylabel = "LPS Endotoxin (pg/mL)", title = "LPS Endotoxin Concentration")

ax2.plot(optimizedSimData[:,0], sims_tnf, label = "Simulated TNF-alpha Data", color = "blue")
ax2.legend(loc = "upper left", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (min)", ylabel = "TNF-alpha (pg/mL)", title = "TNF-alpha Concentration")

ax3.plot(optimizedSimData[:,0], sims_il6, label = "Simulated IL-6 Data", color = "blue")
ax3.legend(loc = "upper left", shadow = True, fancybox = True)
ax3.set(xlabel = "Time (min)", ylabel = "IL-6 (pg/mL)", title = "IL-6 Concentration")

ax4.plot(optimizedSimData[:,0]/60, sims_acth, label = "Simulated ACTH Data", color = "blue")
ax4.legend(loc = "upper left", shadow = True, fancybox = True)
ax4.set(xlabel = "Time (hours)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax5.plot(optimizedSimData[:,0]/60, sims_cort, label = "Simulated Cortisol Data", color = "blue")
ax5.legend(loc = "upper left", shadow = True, fancybox = True)
ax5.set(xlabel = "Time (hours)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

In [ ]:
# plot only HPA axis variables (ACTH and cortisol), if you have set the cytokine initial conditions to 0
%matplotlib inline
fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15,10))

ax1.plot(optimizedSimData[:,0]/60, sims_acth, label = "Simulated ACTH Data", color = "blue")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (hours)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")

ax2.plot(optimizedSimData[:,0]/60, sims_cort, label = "Simulated Cortisol Data", color = "blue")
ax2.legend(loc = "upper left", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (hours)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

plt.savefig("malekModel_output/malekModel-figure3a-reproduction.png", dpi = 300)

[Back to Top](#top)

## Dependencies <a name="dependencies"></a>

In [ ]:
%load_ext watermark

In [ ]:
%watermark --iversions

[Back to Top](#top)